# Automatización Ventas Bsale

### Objetivos :

* ETL para Detalles de ventas, Libros de ventas y Costos. 
* Generación resumen de ventas.

#### Carga de funciones.

In [1]:
import pandas as pd
import numpy as np
import xlsxwriter
import os 
import warnings
import datetime
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

#### Carga de datos

In [2]:
Detalles = pd.read_excel(r"C:\Users\sebit\OneDrive\Documentos\Portafolio\Bsale\Fuentes\Detalle de ventas con atributos - 01_01_2025 - 31_12_2025.xlsx") 
Libro = pd.read_excel(r"C:\Users\sebit\OneDrive\Documentos\Portafolio\Bsale\Fuentes\LIBRO DE VENTA 2025.xlsx", )
Costos = pd.read_excel(r"C:\Users\sebit\OneDrive\Documentos\Portafolio\Bsale\Fuentes\base costos.xlsx")

#### Limpieza Detalles

In [3]:
Detalles.info()
print(Detalles['Tipo de Producto / Servicio'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28274 entries, 0 to 28273
Data columns (total 39 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Tipo Movimiento                            28273 non-null  object 
 1   Tipo de Documento                          28273 non-null  object 
 2   Numero Documento                           28273 non-null  float64
 3   Fecha de Emisión                           28273 non-null  object 
 4   Tracking number                            25033 non-null  object 
 5   Fecha y Hora Venta                         28273 non-null  object 
 6   Sucursal                                   28273 non-null  object 
 7   Vendedor                                   28273 non-null  object 
 8   Nombre Cliente                             28273 non-null  object 
 9   Cliente RUT                                27670 non-null  object 
 10  Email Cliente         

In [4]:
# Eliminacion filas sin datos
Detalles=Detalles.dropna(subset=['Tipo de Documento','Numero Documento'])

# Numero Documento redondeo y SKU a 0 decimales
Detalles['Numero Documento']=Detalles['Numero Documento'].astype(int).round(0)
Detalles['SKU']=Detalles['SKU'].astype(int).round(0)

# Conversiones a string
Conversiones_str=['Tipo de Documento','Numero Documento','Sucursal','SKU']
for col in Conversiones_str:
    Detalles[col]=Detalles[col].astype(str)

# Conversiones int
Conversiones_int=['Precio Neto Unitario','Precio Bruto Unitario','Venta Total Bruta','Cantidad','Venta Total Neta','Venta Total Bruta','Costo neto unitario','Costo Total Neto','Margen', '% Margen']

#Para valores nulos
for col in Conversiones_int:
    Detalles[col]=Detalles[col].fillna(0)

for col in Conversiones_int:
    Detalles[col]=Detalles[col].astype(int)

# Generación Articulo truncando SKU a 8 dígitos
Detalles['Articulo']=Detalles['SKU'].str[:8]

#### Obtener valores costos para utilizarlos en Detalles

In [5]:
# Articulo a str y COSTO UNIT. a int
Costos['ARTICULO']=Costos['ARTICULO'].astype(str)
Costos['COSTO UNIT.']=Costos['COSTO UNIT.'].astype('Int64')

Costos['ARTICULO'] = Costos['ARTICULO'].astype(str).str.strip().str.upper()
Detalles['Articulo'] = Detalles['Articulo'].astype(str).str.strip().str.upper()


In [6]:
type(Costos['COSTO UNIT.'][0])


numpy.int64

In [7]:
Costos=Costos.drop_duplicates(subset=['ARTICULO'], keep='first')
dict_costos=Costos.set_index('ARTICULO')['COSTO UNIT.'].to_dict()

# Mapeo y asignacion de costos faltantes
Condicion = Detalles['Costo neto unitario'].isna() | (Detalles['Costo neto unitario'] == 0)

Detalles.loc[Condicion, 'Costo neto unitario'] = (
    Detalles.loc[Condicion, 'Articulo'].map(dict_costos)
)

Detalles.tail()

,Tipo Movimiento,Tipo de Documento,Numero Documento,Fecha de Emisión,Tracking number,Fecha y Hora Venta,Sucursal,Vendedor,Nombre Cliente,Cliente RUT,Email Cliente,Cliente Dirección,Cliente Comuna,Cliente Ciudad,Lista de Precio,Tipo de entrega,Moneda,Tipo de Producto / Servicio,SKU,Producto / Servicio,Variante,Otros Atributos,Marca,Detalle de Productos/Servicios Pack/Promo,Precio de Lista,Precio Neto Unitario,Precio Bruto Unitario,Cantidad,Venta Total Neta,Total Impuestos,Venta Total Bruta,Nombre de dcto,Descuento Neto,Descuento Bruto,% Descuento,Costo neto unitario,Costo Total Neto,Margen,% Margen,Articulo
28268,venta,BOLETA ELECTRÓNICA T,54051,04/12/2025,695e7117942d391ede8117e2,07/01/2026 11:43:34,Casa Matriz,San Mateo Spa ONLINE,Aurora Ellao,12257359-1,ellaopalma@gmail.com,NaN,NaN,NaN,Lista de Precios Base,retiro en tienda,CLP,Sin Tipo,1767796933252979,Glosa,22601629 Leopardo 38,NaN,NaN,NaN,19995.0,16803,19995,1,16802,3193.0,19995,NaN,0.0,0.0,0.0,NaN,0,16802,1,17677969
28269,venta,BOLETA ELECTRÓNICA T,54051,04/12/2025,695e7117942d391ede8117e2,07/01/2026 11:43:34,Casa Matriz,San Mateo Spa ONLINE,Aurora Ellao,12257359-1,ellaopalma@gmail.com,NaN,NaN,NaN,Lista de Precios Base,retiro en tienda,CLP,Sin Tipo,1767796965252980,Glosa,22601507 Dorado 37,NaN,NaN,NaN,19995.0,16803,19995,1,16803,3192.0,19995,NaN,0.0,0.0,0.0,NaN,0,16803,1,17677969
28270,venta,BOLETA ELECTRÓNICA T,54053,31/12/2025,695e73a4a9445326b6365344,07/01/2026 11:54:27,Casa Matriz,San Mateo Spa ONLINE,Sin cliente,NaN,NaN,NaN,NaN,NaN,Lista de Precios Base,retiro en tienda,CLP,Sin Tipo,1767797555252983,Glosa,22532608 Amarillo 37,NaN,NaN,NaN,9998.0,8402,9998,1,8402,1596.0,9998,NaN,0.0,0.0,0.0,NaN,0,8402,1,17677975
28271,venta,BOLETA ELECTRÓNICA T,54053,31/12/2025,695e73a4a9445326b6365344,07/01/2026 11:54:27,Casa Matriz,San Mateo Spa ONLINE,Sin cliente,NaN,NaN,NaN,NaN,NaN,Lista de Precios Base,retiro en tienda,CLP,Sin Tipo,1767797581252984,Glosa,22542611 Negro 38,NaN,NaN,NaN,8998.0,7561,8998,1,7561,1437.0,8998,NaN,0.0,0.0,0.0,NaN,0,7561,1,17677975
28272,venta,BOLETA ELECTRÓNICA T,54053,31/12/2025,695e73a4a9445326b6365344,07/01/2026 11:54:27,Casa Matriz,San Mateo Spa ONLINE,Sin cliente,NaN,NaN,NaN,NaN,NaN,Lista de Precios Base,retiro en tienda,CLP,Sin Tipo,1767797610252985,Glosa,22542611 Camel 38,NaN,NaN,NaN,8998.0,7561,8998,1,7561,1437.0,8998,NaN,0.0,0.0,0.0,NaN,0,7561,1,17677976


#### Obtener Marketplace a partir de Libro de ventas

In [8]:
Libro.head(12)

,Cliente,Unnamed: 1,Unnamed: 2,Todos,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60
0,Fecha de Inicio,NaN,NaN,2025-12-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fecha de Término,NaN,NaN,2025-12-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tipo Documento,NaN,NaN,Todos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Nº Documento,NaN,NaN,Todos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sucursal,NaN,NaN,Todos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Vendedor,NaN,NaN,Todos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Emisor,NaN,NaN,Todos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Otros Atributos,NaN,NaN,Todos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,BOLETA ELECTRÓNICA T (1850),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def detectar_y_separar_tablas(df_raw, headers_buscar=['tipo documento', 'nº documento', 'rut cliente']):
    """
    Detecta headers y crea una tabla por cada tipo de documento encontrado.
    Cada tabla comienza con un header y termina cuando cambia el tipo documento.
    
    Parameters:
    -----------
    df_raw : DataFrame
        DataFrame leído sin header (header=None)
    headers_buscar : list
        Lista de valores de header a buscar (en minúsculas)
    
    Returns:
    --------
    list : Lista de DataFrames, una por cada tabla encontrada
    """
    # Eliminar filas vacías Y RESETEAR ÍNDICES
    df_raw = df_raw.dropna(how='all').reset_index(drop=True)
    
    tables = []
    
    # Convertir todo a string y minúsculas para búsqueda
    df_search = df_raw.astype(str).apply(lambda x: x.str.lower().str.strip())
    
    # Buscar filas que contienen los headers
    header_indices = []
    for idx, row in df_search.iterrows():
        headers_encontrados = sum([any(h in str(val) for val in row.values) for h in headers_buscar])
        if headers_encontrados >= 2:
            header_indices.append(idx)
    
    if not header_indices:
        return []
    
    # Procesar cada header encontrado
    for i, header_row in enumerate(header_indices):
        # Obtener el header
        header_names = df_raw.iloc[header_row].astype(str).str.strip()
        
        # Encontrar dónde termina esta sección (siguiente header o final)
        if i + 1 < len(header_indices):
            max_row = header_indices[i + 1]
        else:
            max_row = len(df_raw)
        
        # Extraer datos desde después del header
        data_start = header_row + 1
        section_data = df_raw.iloc[data_start:max_row].copy()
        
        if section_data.empty:
            continue
        
        # Asignar nombres de columnas
        section_data.columns = header_names
        section_data = section_data.reset_index(drop=True)
        
        # Normalizar nombres de columnas
        section_data.columns = section_data.columns.astype(str).str.strip().str.lower()
        
        # Eliminar filas completamente vacías
        section_data = section_data.dropna(how='all').reset_index(drop=True)
        
        if section_data.empty:
            continue
        
        # Identificar columna de tipo documento
        tipo_doc_col = None
        for col in section_data.columns:
            col_clean = str(col).lower().strip()
            if 'tipo' in col_clean and ('doc' in col_clean or 'documento' in col_clean):
                tipo_doc_col = col
                break
        
        if tipo_doc_col is None:
            continue
        
        # Limpiar columna tipo documento
        section_data[tipo_doc_col] = section_data[tipo_doc_col].astype(str).str.strip().str.upper()
        
        # Eliminar filas sin tipo documento
        section_data = section_data[
            (section_data[tipo_doc_col] != '') & 
            (section_data[tipo_doc_col] != 'NAN') &
            (section_data[tipo_doc_col].notna())
        ].reset_index(drop=True)
        
        if section_data.empty:
            continue
        
        # Obtener el primer tipo de documento
        primer_tipo = section_data[tipo_doc_col].iloc[0]
        
        # Encontrar dónde cambia el tipo documento
        tipo_cambios = section_data[tipo_doc_col] != primer_tipo
        
        if tipo_cambios.any():
            # Hay un cambio - tomar solo hasta el cambio
            primer_cambio = tipo_cambios.idxmax()
            tabla = section_data.iloc[:primer_cambio].copy()
        else:
            # No hay cambio - tomar toda la sección
            tabla = section_data.copy()
        
        # Limpiar filas de totales al final
        if len(tabla) > 2:
            last_rows = tabla.tail(3).astype(str).apply(lambda x: x.str.lower())
            mask_total = last_rows.apply(
                lambda row: any('total' in str(val) or 'suma' in str(val) for val in row), 
                axis=1
            )
            if mask_total.any():
                rows_to_drop = mask_total[mask_total].index
                tabla = tabla.drop(rows_to_drop)
        
        tabla = tabla.reset_index(drop=True)
        
        if not tabla.empty:
            tables.append(tabla)
    
    return tables


def procesar_libro_completo(ruta_archivo):
    """
    Procesa todas las hojas del libro de ventas Excel.
    
    Parameters:
    -----------
    ruta_archivo : str
        Ruta al archivo Excel
    
    Returns:
    --------
    dict : Diccionario con hojas como keys y listas de tablas como values
    """
    print("\n" + "="*70)
    print("CARGANDO ARCHIVO EXCEL")
    print("="*70)
    
    # Leer todas las hojas
    try:
        todas_hojas = pd.read_excel(ruta_archivo, sheet_name=None, header=None)
        print(f"Archivo cargado exitosamente")
        print(f"Hojas encontradas: {len(todas_hojas)}")
        for nombre in todas_hojas.keys():
            print(f"   • {nombre}")
    except Exception as e:
        print(f"Error al cargar archivo: {e}")
        return {}
    
    # Procesar cada hoja
    resultados = {}
    
    for nombre_hoja, df_hoja in todas_hojas.items():
        print("\n" + "="*70)
        print(f"PROCESANDO HOJA: {nombre_hoja}")
        print("="*70)
        
        tablas = detectar_y_separar_tablas(df_hoja)
        
        if tablas:
            resultados[nombre_hoja] = tablas
            print(f"{len(tablas)} tabla(s) encontrada(s)")
            
            # Mostrar resumen de esta hoja
            for idx, tabla in enumerate(tablas, 1):
                tipo_col = [c for c in tabla.columns if 'tipo' in c and ('doc' in c or 'documento' in c)]
                if tipo_col:
                    tipo_unico = tabla[tipo_col[0]].unique()[0]
                    print(f"   Tabla {idx}: {len(tabla):4d} filas | Tipo: {tipo_unico}")
        else:
            print(f"No se encontraron tablas en esta hoja")
    
    return resultados


# ===== EJECUCIÓN =====
# Actualiza esta ruta con la ubicación de tu archivo
ruta_libro = r"C:\Users\sebit\OneDrive\Documentos\Portafolio\Bsale\Fuentes\LIBRO DE VENTAS 2024.xlsx"

# Procesar todas las hojas
todas_tablas = procesar_libro_completo(ruta_libro)

# ===== RESUMEN FINAL =====
print("\n" + "="*70)
print("RESUMEN FINAL")
print("="*70)

total_tablas = sum(len(tablas) for tablas in todas_tablas.values())
print(f"\nTotal de hojas procesadas: {len(todas_tablas)}")
print(f"Total de tablas encontradas: {total_tablas}")

for nombre_hoja, tablas in todas_tablas.items():
    print(f"\n{nombre_hoja}:")
    print(f"   {len(tablas)} tabla(s)")
    
    for idx, tabla in enumerate(tablas, 1):
        tipo_col = [c for c in tabla.columns if 'tipo' in c and ('doc' in c or 'documento' in c)]
        if tipo_col:
            tipo_unico = tabla[tipo_col[0]].unique()[0]
            
            # Buscar marketplace si existe
            marketplace_col = [c for c in tabla.columns if 'marketplace' in c.lower()]
            
            print(f"\n   Tabla {idx}:")
            print(f"      Filas: {len(tabla)}")
            print(f"      Tipo: {tipo_unico}")
            
            if marketplace_col and len(marketplace_col) > 0:
                marketplaces = tabla[marketplace_col[0]].value_counts()
                if not marketplaces.empty:
                    print(f"      Marketplaces:")
                    for mp, count in marketplaces.items():
                        if str(mp).upper() != 'NAN':
                            print(f"         • {mp}: {count} docs")

print("\n" + "="*70)

# ===== ACCESO A LAS TABLAS =====
# Ejemplo de cómo acceder a las tablas:
# todas_tablas['Hoja1'][0]  -> Primera tabla de Hoja1
# todas_tablas['Hoja2'][1]  -> Segunda tabla de Hoja2

# Mostrar preview de la primera tabla encontrada
if todas_tablas:
    primera_hoja = list(todas_tablas.keys())[0]
    primera_tabla = todas_tablas[primera_hoja][0]
    
    print("\nPREVIEW - PRIMERA TABLA ENCONTRADA:")
    print(f"Hoja: {primera_hoja}")
    print("-"*70)
    print(primera_tabla.head(5))
    print("-"*70)
    print(f"Total de columnas: {len(primera_tabla.columns)}")
    print(f"Nombres de columnas: {list(primera_tabla.columns[:10])}...")


CARGANDO ARCHIVO EXCEL
Archivo cargado exitosamente
Hojas encontradas: 4
   • BS12
   • DPCache_bs01
   • DPCache_bs01 -1
   • DPCache_bs01 0

PROCESANDO HOJA: BS12
No se encontraron tablas en esta hoja

PROCESANDO HOJA: DPCache_bs01
No se encontraron tablas en esta hoja

PROCESANDO HOJA: DPCache_bs01 -1
No se encontraron tablas en esta hoja

PROCESANDO HOJA: DPCache_bs01 0
No se encontraron tablas en esta hoja

RESUMEN FINAL

Total de hojas procesadas: 0
Total de tablas encontradas: 0



In [10]:
import pandas as pd
import numpy as np
from collections import Counter

def crear_diccionarios_documentos_v2(todas_tablas):
    
    dict_marketplace = {}
    dict_notas_credito = {}
    
    estadisticas = {
        'total_documentos_procesados': 0,
        'documentos_con_marketplace': 0,
        'notas_credito_encontradas': 0,
        'notas_credito_con_nota': 0,
        'documentos_sin_marketplace': 0,
        'duplicados_encontrados': 0,
        'hojas_procesadas': 0,
        'tablas_procesadas': 0
    }
    
    # Variantes de nombres de columnas
    variantes_nro_doc = ['nº documento', 'n documento', 'numero documento', 'nro documento']
    variantes_marketplace = ['marketplace', 'market place', 'canal']
    variantes_tipo_doc = ['tipo documento', 'tipo de documento', 'tipo doc']
    variantes_nota = ['nota', 'observacion', 'observación', 'obser.', 'obser']
    
    def buscar_columna(df, variantes):
        """Busca una columna por sus variantes y devuelve el índice"""
        cols_lower = [(i, col.lower().strip()) for i, col in enumerate(df.columns)]
        for variante in variantes:
            for idx, col_lower in cols_lower:
                if variante.lower() == col_lower:
                    return idx
        return None
    
    print("\n" + "="*70)
    print("CREANDO DICCIONARIOS DE DOCUMENTOS (V2)")
    print("="*70)
    
    # Procesar cada hoja
    for nombre_hoja, tablas in todas_tablas.items():
        print(f"\nProcesando hoja: {nombre_hoja}")
        estadisticas['hojas_procesadas'] += 1
        
        # Procesar cada tabla
        for idx_tabla, tabla in enumerate(tablas, 1):
            estadisticas['tablas_procesadas'] += 1
            
            # CRÍTICO: Eliminar columnas duplicadas
            tabla_limpia = tabla.loc[:, ~tabla.columns.duplicated(keep='first')].copy()
            
            # Buscar índices de columnas
            idx_nro_doc = buscar_columna(tabla_limpia, variantes_nro_doc)
            idx_marketplace = buscar_columna(tabla_limpia, variantes_marketplace)
            idx_tipo_doc = buscar_columna(tabla_limpia, variantes_tipo_doc)
            idx_nota = buscar_columna(tabla_limpia, variantes_nota)
            
            if idx_nro_doc is None:
                print(f"  Tabla {idx_tabla}: No se encontró 'Nº Documento'")
                continue
            
            print(f"  Tabla {idx_tabla}: {len(tabla_limpia)} documentos")
            
            # Convertir a numpy array para acceso más rápido y seguro
            datos = tabla_limpia.values
            
            # Procesar cada fila
            for i in range(len(datos)):
                # Acceder directamente por índice de columna
                nro_doc = str(datos[i, idx_nro_doc]).strip()
                
                # Validar
                if nro_doc == '' or nro_doc.upper() == 'NAN':
                    continue
                
                estadisticas['total_documentos_procesados'] += 1
                
                # ==== MARKETPLACE ====
                if idx_marketplace is not None:
                    marketplace = str(datos[i, idx_marketplace]).strip()
                    
                    if marketplace != '' and marketplace.upper() != 'NAN':
                        if nro_doc in dict_marketplace:
                            if dict_marketplace[nro_doc] != marketplace:
                                estadisticas['duplicados_encontrados'] += 1
                        else:
                            dict_marketplace[nro_doc] = marketplace
                            estadisticas['documentos_con_marketplace'] += 1
                    else:
                        estadisticas['documentos_sin_marketplace'] += 1
                else:
                    estadisticas['documentos_sin_marketplace'] += 1
                
                # ==== NOTAS DE CRÉDITO ====
                if idx_tipo_doc is not None:
                    tipo_doc = str(datos[i, idx_tipo_doc]).strip().upper()
                    
                    if 'NOTA' in tipo_doc and 'CRÉDITO' in tipo_doc:
                        estadisticas['notas_credito_encontradas'] += 1
                        
                        if idx_nota is not None:
                            nota = str(datos[i, idx_nota]).strip()
                            
                            if nota != '' and nota.upper() != 'NAN':
                                dict_notas_credito[nro_doc] = nota
                                estadisticas['notas_credito_con_nota'] += 1
    
    # Verificación
    print("\n" + "="*70)
    print("VERIFICACIÓN DE DICCIONARIOS")
    print("="*70)
    
    if dict_marketplace:
        print("\nPrimeros 5 en dict_marketplace:")
        for doc, mp in list(dict_marketplace.items())[:5]:
            print(f"  Doc: '{doc}' ({len(doc)} chars) -> Marketplace: '{mp}'")
    
    if dict_notas_credito:
        print("\nPrimeros 5 en dict_notas_credito:")
        for doc, nota in list(dict_notas_credito.items())[:5]:
            nota_prev = nota[:40] + "..." if len(nota) > 40 else nota
            print(f"  Doc: '{doc}' ({len(doc)} chars) -> Nota: '{nota_prev}'")
    
    return dict_marketplace, dict_notas_credito, estadisticas


def mostrar_resumen_diccionarios(dict_marketplace, dict_notas_credito, estadisticas):
    
    print("\n" + "="*70)
    print("RESUMEN DE DICCIONARIOS")
    print("="*70)
    
    print(f"\nESTADÍSTICAS:")
    print(f"   Hojas procesadas: {estadisticas['hojas_procesadas']}")
    print(f"   Tablas procesadas: {estadisticas['tablas_procesadas']}")
    print(f"   Documentos procesados: {estadisticas['total_documentos_procesados']}")
    print(f"   Duplicados: {estadisticas['duplicados_encontrados']}")
    
    print(f"\nMARKETPLACE:")
    print(f"   Documentos con marketplace: {len(dict_marketplace)}")
    print(f"   Documentos sin marketplace: {estadisticas['documentos_sin_marketplace']}")
    
    if dict_marketplace:
        mp_count = Counter(dict_marketplace.values())
        print(f"\n   Distribución:")
        for mp, count in mp_count.most_common():
            print(f"      • {mp}: {count} docs")
    
    print(f"\nNOTAS DE CRÉDITO:")
    print(f"   Total NC encontradas: {estadisticas['notas_credito_encontradas']}")
    print(f"   NC con nota: {len(dict_notas_credito)}")
    print(f"   NC sin nota: {estadisticas['notas_credito_encontradas'] - len(dict_notas_credito)}")
    
    print("\n" + "="*70)


def guardar_diccionarios(dict_marketplace, dict_notas_credito, ruta_salida=None):
    import os
    from datetime import datetime
    
    if ruta_salida is None:
        ruta_salida = r"C:\Users\sebit\OneDrive\Documentos\Portafolio\Bsale\Procesados"
    
    os.makedirs(ruta_salida, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    print("\n" + "="*70)
    print("GUARDANDO DICCIONARIOS")
    print("="*70)
    
    if dict_marketplace:
        df_mp = pd.DataFrame(list(dict_marketplace.items()), 
                            columns=['Nro_Documento', 'Marketplace'])
        archivo_mp = os.path.join(ruta_salida, f'dict_marketplace_{timestamp}.xlsx')
        df_mp.to_excel(archivo_mp, index=False)
        print(f"Marketplace: {archivo_mp}")
    
    if dict_notas_credito:
        df_nc = pd.DataFrame(list(dict_notas_credito.items()), 
                            columns=['Nro_Documento', 'Nota'])
        archivo_nc = os.path.join(ruta_salida, f'dict_notas_credito_{timestamp}.xlsx')
        df_nc.to_excel(archivo_nc, index=False)
        print(f"Notas crédito: {archivo_nc}")
    
    print("="*70)


# ===== EJECUTAR =====
print("\nRECREANDO DICCIONARIOS CON V2...")

dict_marketplace, dict_notas_credito, stats = crear_diccionarios_documentos_v2(todas_tablas)

# Mostrar resumen
mostrar_resumen_diccionarios(dict_marketplace, dict_notas_credito, stats)

# Guardar
guardar_diccionarios(dict_marketplace, dict_notas_credito)

# ===== APLICAR A DETALLES =====
print("\n" + "="*70)
print("APLICANDO A DETALLES")
print("="*70)

Detalles['Marketplace'] = Detalles['Numero Documento'].astype(str).str.strip().map(dict_marketplace)
Detalles['Nota'] = Detalles['Numero Documento'].astype(str).str.strip().map(dict_notas_credito)

docs_con_mp = Detalles['Marketplace'].notna().sum()


print(f"\nRESULTADOS:")
print(f"   Con marketplace: {docs_con_mp}/{len(Detalles)} ({docs_con_mp/len(Detalles)*100:.1f}%)")


if docs_con_mp > 0:
    print("\nDistribución marketplaces en Detalles:")
    for mp, count in Detalles['Marketplace'].value_counts().items():
        if pd.notna(mp):
            print(f"   • {mp}: {count}")

# Verificación final
print("\nVERIFICACIÓN FINAL:")
print("-"*70)
print("\nCON marketplace:")
print(Detalles[Detalles['Marketplace'].notna()][['Numero Documento', 'Tipo de Documento', 'Marketplace']].head(10))


print("-"*70)


RECREANDO DICCIONARIOS CON V2...

CREANDO DICCIONARIOS DE DOCUMENTOS (V2)

Procesando hoja: BSene24
  Tabla 1: 1937 documentos
  Tabla 2: 104 documentos
  Tabla 3: 643 documentos

Procesando hoja: BSfeb24
  Tabla 1: 1302 documentos
  Tabla 2: 25 documentos
  Tabla 3: 368 documentos

Procesando hoja: BS marz24
  Tabla 1: 2284 documentos
  Tabla 2: 47 documentos
  Tabla 3: 104 documentos
  Tabla 4: 458 documentos

Procesando hoja: BS04-24
  Tabla 1: 949 documentos
  Tabla 2: 11 documentos
  Tabla 3: 93 documentos
  Tabla 4: 321 documentos
  Tabla 5: 3 documentos

Procesando hoja: BS05-24
  Tabla 1: 1147 documentos
  Tabla 2: 7 documentos
  Tabla 3: 92 documentos
  Tabla 4: 175 documentos
  Tabla 5: 1 documentos

Procesando hoja: BS06-24
  Tabla 1: 1681 documentos
  Tabla 2: 1 documentos
  Tabla 3: 82 documentos
  Tabla 4: 281 documentos

Procesando hoja: BS07-24
  Tabla 1: 787 documentos
  Tabla 2: 1 documentos
  Tabla 3: 2 documentos
  Tabla 4: 47 documentos
  Tabla 5: 213 documentos



In [11]:
def generar_excel_formateado(df_detalles, ruta_salida=None):
    """
    Genera Excel con formato bonito y múltiples hojas de resumen.
    """
    
    if ruta_salida is None:
        ruta_salida = r"C:\Users\sebit\OneDrive\Documentos\Portafolio\Bsale\Procesados"
    
    os.makedirs(ruta_salida, exist_ok=True)
    archivo = os.path.join(ruta_salida, f'Ventas Automatizado.xlsx')
    
    print("\n" + "="*70)
    print("GENERANDO EXCEL FORMATEADO")
    print("="*70)
    
    # ===== CREAR WORKBOOK =====
    workbook = xlsxwriter.Workbook(archivo, {'nan_inf_to_errors': True})
    
    # ===== DEFINIR FORMATOS =====
    header_format = workbook.add_format({
        'bg_color': '#1F4E78',
        'font_color': 'white',
        'bold': True,
        'border': 1,
        'align': 'center',
        'valign': 'vcenter',
        'text_wrap': True
    })
    
    datos_format = workbook.add_format({
        'border': 1,
        'align': 'left',
        'valign': 'vcenter'
    })
    
    # FORMATOS SIN DECIMALES
    numero_format = workbook.add_format({
        'border': 1,
        'align': 'right',
        'valign': 'vcenter',
        'num_format': '#,##0'  # SIN decimales
    })
    
    # FORMATOS CON DECIMALES
    decimal_format = workbook.add_format({
        'border': 1,
        'align': 'right',
        'valign': 'vcenter',
        'num_format': '#,##0.00'
    })
    
    porcentaje_format = workbook.add_format({
        'border': 1,
        'align': 'right',
        'valign': 'vcenter',
        'num_format': '0.00"%"'
    })
    
    # FORMATOS PARA TOTALES
    total_numero_format = workbook.add_format({
        'bg_color': '#D9E1F2',
        'bold': True,
        'border': 1,
        'align': 'right',
        'valign': 'vcenter',
        'num_format': '#,##0'  # SIN decimales
    })
    
    total_decimal_format = workbook.add_format({
        'bg_color': '#D9E1F2',
        'bold': True,
        'border': 1,
        'align': 'right',
        'valign': 'vcenter',
        'num_format': '#,##0.00'
    })
    
    total_porcentaje_format = workbook.add_format({
        'bg_color': '#D9E1F2',  
        'bold': True,
        'border': 1,
        'align': 'right',
        'valign': 'vcenter',
        'num_format': '0.00"%"'
    })
    
    # ===== HOJA 1: DETALLES =====
    worksheet_detalles = workbook.add_worksheet('Detalles')
    
    df_export = df_detalles.copy()
    df_export = df_export.fillna(0)
    df_export = df_export.replace([np.inf, -np.inf], 0)
    
    # Definir qué columnas llevan decimales y cuáles no
    columnas_sin_decimales = ['Cantidad', 'Precio Neto Unitario', 'Precio Bruto Unitario',
                               'Costo neto unitario', 'Numero Documento', 'Venta Total Neta', 'Total Impuestos', 'Venta Total Bruta', 
                              'Costo Total Neto', 'Margen', 'Precio de Lista']
    columnas_con_decimales = ['Descuento Neto', 'Descuento Bruto']
    columnas_porcentaje = ['% Margen', '% Descuento']
    
    # Escribir headers
    for col_num, valor in enumerate(df_export.columns):
        worksheet_detalles.write(0, col_num, valor, header_format)
    
    # Ajustar ancho de columnas
    for col_num, col_name in enumerate(df_export.columns):
        ancho = max(
            len(str(col_name)) + 2,
            df_export[col_name].astype(str).str.len().max() + 1
        )
        ancho = min(ancho, 40)
        worksheet_detalles.set_column(col_num, col_num, ancho)
    
    # Escribir datos
    for row_num, row_data in enumerate(df_export.values, 1):
        for col_num, valor in enumerate(row_data):
            col_name = df_export.columns[col_num]
            
            if col_name in columnas_sin_decimales:
                worksheet_detalles.write(row_num, col_num, valor, numero_format)
            elif col_name in columnas_con_decimales:
                worksheet_detalles.write(row_num, col_num, valor, decimal_format)
            elif col_name in columnas_porcentaje:
                worksheet_detalles.write(row_num, col_num, valor, porcentaje_format)
            else:
                worksheet_detalles.write(row_num, col_num, valor, datos_format)
    
    worksheet_detalles.freeze_panes(1, 0)
    
    print(f"\nHoja 'Detalles': {len(df_export):,} filas x {len(df_export.columns)} columnas")
    
    # ===== FUNCIÓN AUXILIAR PARA CREAR HOJAS DE RESUMEN =====
    def crear_hoja_resumen(df, columna_agrupacion, nombre_hoja):
        """
        Crea una hoja de resumen agrupada por una columna específica.
        """
        worksheet = workbook.add_worksheet(nombre_hoja)
        
        # Verificar si existe la columna
        if columna_agrupacion not in df.columns:
            print(f"Advertencia: No se encontró la columna '{columna_agrupacion}'")
            return
        
        # Agrupar datos
        resumen = df.groupby(columna_agrupacion).agg({
            'Numero Documento': 'count',
            'Cantidad': 'sum',
            'Venta Total Neta': 'sum',
            'Costo Total Neto': 'sum',
            'Margen': 'sum'
        }).fillna(0)
        
        resumen.columns = ['Documentos', 'Cantidad Total', 'Venta Total Neta', 'Costo Total Neto', 'Margen Total']
        
        # Calcular % Margen
        resumen['% Margen Promedio'] = np.where(
            resumen['Venta Total Neta'] != 0,
            (resumen['Margen Total'] / resumen['Venta Total Neta'] * 100),
            0
        )
        
        resumen = resumen.sort_values('Venta Total Neta', ascending=False)
        
        # Definir columnas del resumen que no deben mostrar decimales
        columnas_sin_decimales_resumen = ['Documentos', 'Cantidad Total', 'Venta Total Neta', 'Costo Total Neto', 'Margen Total']
        columnas_porcentaje_resumen = ['% Margen Promedio']
        
        # Escribir headers
        worksheet.write(0, 0, columna_agrupacion, header_format)
        for col_num, valor in enumerate(resumen.columns, 1):
            worksheet.write(0, col_num, valor, header_format)
        
        # Escribir datos
        for row_num, (categoria, row_data) in enumerate(resumen.iterrows(), 1):
            worksheet.write(row_num, 0, str(categoria), datos_format)
            
            for col_num, (col_name, valor) in enumerate(zip(resumen.columns, row_data.values), 1):
                # SIN decimales para columnas definidas (Documentos, Cantidad y totales monetarios)
                if col_name in columnas_sin_decimales_resumen:
                    worksheet.write_number(row_num, col_num, int(round(valor)), numero_format)
                # Porcentaje para % Margen Promedio
                elif col_name in columnas_porcentaje_resumen:
                    worksheet.write_number(row_num, col_num, float(valor), porcentaje_format)
                # CON decimales para valores no incluidos en columnas_sin_decimales_resumen
                else:
                    worksheet.write_number(row_num, col_num, float(valor), decimal_format)
        
        # Ajustar ancho de columnas
        worksheet.set_column(0, 0, 30)
        for col_num in range(1, len(resumen.columns) + 1):
            worksheet.set_column(col_num, col_num, 18)
        
        # ===== FILA DE TOTALES =====
        fila_total = len(resumen) + 1
        worksheet.write(fila_total, 0, 'TOTAL', header_format)
        
        # Columnas para sumar (todas excepto % Margen que se calcula)
        columnas_sumar = ['Documentos', 'Cantidad Total', 'Venta Total Neta', 
                         'Costo Total Neto', 'Margen Total']
        
        for col_num, col_name in enumerate(resumen.columns, 1):
            if col_name in columnas_sumar:
                # Calcular el total de Python directamente (más confiable que fórmulas)
                total = resumen[col_name].sum()
                
                # Usar formato entero para columnas sin decimales en resumen
                if col_name in columnas_sin_decimales_resumen:
                    worksheet.write_number(fila_total, col_num, int(round(total)), total_numero_format)
                else:
                    worksheet.write_number(fila_total, col_num, float(total), total_decimal_format)
            
            elif col_name == '% Margen Promedio':
                # Calcular % Margen Total = (Margen Total / Venta Total) * 100
                venta_total = resumen['Venta Total Neta'].sum()
                margen_total = resumen['Margen Total'].sum()
                pct_margen = (margen_total / venta_total * 100) if venta_total != 0 else 0
                worksheet.write_number(fila_total, col_num, float(pct_margen), total_porcentaje_format)
        
        print(f"Hoja '{nombre_hoja}': {len(resumen)} categorías")
        return resumen
    
    # ===== HOJA 2: RESUMEN POR MARKETPLACE =====
    crear_hoja_resumen(df_export, 'Marketplace', 'Resumen Marketplace')
    
    # ===== HOJA 3: RESUMEN POR TIPO DE PRODUCTO =====
    crear_hoja_resumen(df_export, 'Tipo de Producto / Servicio', 'Resumen Tipo Producto')
    
    # ===== HOJA 4: RESUMEN POR SUCURSAL =====
    crear_hoja_resumen(df_export, 'Sucursal', 'Resumen Sucursal')
    
    # ===== HOJA 5: RESUMEN POR VENDEDOR =====
    crear_hoja_resumen(df_export, 'Vendedor', 'Resumen Vendedor')
    
    # ===== CERRAR WORKBOOK =====
    workbook.close()
    
    print(f"\nArchivo guardado en:")
    print(f"   {archivo}")
    print("="*70)
    
    return archivo


# ===== EJECUTAR =====
archivo_excel = generar_excel_formateado(Detalles)
print(f"\nExcel generado: {archivo_excel}")


GENERANDO EXCEL FORMATEADO

Hoja 'Detalles': 28,273 filas x 42 columnas
Hoja 'Resumen Marketplace': 1 categorías
Hoja 'Resumen Tipo Producto': 7 categorías
Hoja 'Resumen Sucursal': 2 categorías
Hoja 'Resumen Vendedor': 4 categorías

Archivo guardado en:
   C:\Users\sebit\OneDrive\Documentos\Portafolio\Bsale\Procesados\Ventas Automatizado.xlsx

Excel generado: C:\Users\sebit\OneDrive\Documentos\Portafolio\Bsale\Procesados\Ventas Automatizado.xlsx
